# Fine-grained Arabic Named Entity Recognition

# Bi-LSTM Model

# Load Notebook Environment
Load the pretrained kernel and environment without having to rerun all cells.

In [ ]:
import dill
dill.load_session('notebookWorking_env.db')

# Import Libraries

In [1]:
import ast

import numpy as np

import codecs

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import InputLayer, Embedding, LSTM, Dense, Activation

from keras.optimizers import Adam

from keras.preprocessing.sequence import pad_sequences

from keras.utils import to_categorical

2024-06-13 09:47:54.967130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Read/Processes Training Data

In [21]:
sentences = []
labels = []

curr_sentence = []
curr_labels = []
final_lines = []
final = ""
with open("/Users/ziadhamwi/Desktop/TeacherStudent/TrainingG_Data.txt", "r") as file:
    for line in file:
        if line != "\n":
            label, word = line.split()[:2]
            
            final_lines.append(f"{label} {word}\n")
            
            curr_sentence.append(word)
            curr_labels.append(label)
        else:
            final_lines.append("\n")
            sentences.append(curr_sentence)
            labels.append(curr_labels)
            curr_sentence = []
            curr_labels = []
            
final = "".join(final_lines)

print("DONE!")


DONE!


In [22]:
print(len(final))

9768817


In [23]:
new_format = []
sentence = []

output2023 = ["["]
lines = final.split('\n')
drop = len(final.split('\n'))

for i in range(drop):
    curr_line = lines[i]
    if len(curr_line) != 0:
        parts = curr_line.split()
        subentity = parts[0]
        word = parts[1]

        if word != ".":
            output2023.append(f"('{word}', '{subentity}')")
            if i != drop-1:  
                output2023.append(", ")
        else:
            output2023.append("('.', 'OUTSIDE')]\n[")
            
if output2023[-1] == "[":
    output2023.pop()
else:
    if output2023[-1] == ", ":
        output2023.pop()

output2023.append("]")

output2023 = "".join(output2023)


print("Done!")


Done!


In [24]:
training_sentences = []
for line in output2023[:-3].split("\n"):
    training_sentences.append(line+"\n")
    
print(len(training_sentences))


27200


# Read/Processes Testing Data

In [25]:
sentences = []
labels = []

curr_sentence = []
curr_labels = []
final_lines = []
final = ""
with open("/Users/ziadhamwi/Desktop/TeacherStudent/TestingData.txt", "r") as file:
    for line in file:
        if line != "\n":
            label, word = line.split()[:2]
            
            final_lines.append(f"{label} {word}\n")
            
            curr_sentence.append(word)
            curr_labels.append(label)
        else:
            final_lines.append("\n")
            sentences.append(curr_sentence)
            labels.append(curr_labels)
            curr_sentence = []
            curr_labels = []
            
final = "".join(final_lines)

print("DONE!")


DONE!


In [26]:
print(len(final))

306023


In [27]:
new_format = []
sentence = []

output2023 = ["["]
lines = final.split('\n')
drop = len(final.split('\n'))

for i in range(drop):
    curr_line = lines[i]
    if len(curr_line) != 0:
        parts = curr_line.split()
        subentity = parts[0]
        word = parts[1]

        if word != ".":
            output2023.append(f"('{word}', '{subentity}')")
            if i != drop-1:
                output2023.append(", ")
        else:
            output2023.append("('.', 'OUTSIDE')]\n[")
            
if output2023[-1] == "[":
    output2023.pop()
else:
    if output2023[-1] == ", ":
        output2023.pop()

output2023.append("]")

output2023 = "".join(output2023)


print("Done!")


Done!


In [28]:
testing_sentences = []
for line in output2023[:-3].split("\n"):
    testing_sentences.append(line+"\n")
    
print(len(testing_sentences))


856


# Preview data

In [29]:
# Display a few rows of the dataset
num_rows = 5  # Number of rows to display
print("\n\nTraining Set")
for i in range(num_rows):
    print(training_sentences[i])

print("\n\nTesting Set")
for i in range(num_rows):
    print(testing_sentences[i])



Training Set
[('إنجلترا', 'B-Country'), ('أو', 'OUTSIDE'), ('إنكلترا', 'B-Country'), ('أو', 'OUTSIDE'), ('إنكلترة', 'B-Country'), ('بالإنجليزية', 'B-Language'), ('إنگلاند', 'OUTSIDE'), ('هي', 'OUTSIDE'), ('أكبر', 'OUTSIDE'), ('دولة', 'OUTSIDE'), ('في', 'OUTSIDE'), ('المملكة', 'B-Country'), ('المتحدة', 'I-Country'), ('وتشترك', 'OUTSIDE'), ('في', 'OUTSIDE'), ('الحدود', 'OUTSIDE'), ('البرية', 'OUTSIDE'), ('مع', 'OUTSIDE'), ('اسكتلندا', 'B-Country'), ('في', 'OUTSIDE'), ('الشمال', 'OUTSIDE'), ('وويلز', 'OUTSIDE'), ('في', 'OUTSIDE'), ('الغرب', 'OUTSIDE'), ('والبحر', 'B-Body_Of_Water'), ('الأيرلندي', 'I-Body_Of_Water'), ('في', 'OUTSIDE'), ('الشمال', 'OUTSIDE'), ('الغربي', 'OUTSIDE'), ('وبحر', 'B-Body_Of_Water'), ('الكلت', 'I-Body_Of_Water'), ('في', 'OUTSIDE'), ('الجنوب', 'OUTSIDE'), ('الغربي', 'OUTSIDE'), ('وبحر', 'B-Body_Of_Water'), ('الشمال', 'I-Body_Of_Water'), ('في', 'OUTSIDE'), ('الشرق', 'OUTSIDE'), ('وتفصلها', 'OUTSIDE'), ('القناة', 'OUTSIDE'), ('الإنجليزية', 'B-Language'), ('عن', 'OU

# NLP Data Preprocessing Functions for Neural Network Training

In [30]:
def get_words(sentences):
    return set(word for sentence in sentences for word in sentence)

def get_tags(sentence_tags):
    return set(tag for tags in sentence_tags for tag in tags)

def get_train_sentences_x(sentences, word2index):
    return [[word2index[word] for word in sentence] for sentence in sentences]

def get_test_sentences_x(sentences, word2index):
    return [[word2index.get(word, word2index['-OOV-']) for word in sentence] for sentence in sentences]

def get_train_tags_y(tags, tag2index):
    return [[tag2index[tag] for tag in tag_sequence] for tag_sequence in tags]

def get_test_tags_y(tags, tag2index):
    return [[tag2index[tag] for tag in tag_sequence] for tag_sequence in tags]

# Converting Logits to Tokens in Natural Language Processing

In [31]:
# Function to convert logits to tokens
def logits_to_tokens(predictions, index_to_tag):
    return [index_to_tag[np.argmax(prediction)] for prediction in predictions]

# Extracting Sentences and Tags from Tagged Sentences in Natural Language Processing

In [32]:
# Extract sentences and tags from tagged sentences -> Training Sentences/Tags
sentences_training, training_sentence_tags = [], []
for tagged_sentence in training_sentences:
    try:
        sentence, tags = zip(*ast.literal_eval(tagged_sentence.strip()))
        sentences_training.append(list(sentence))  # Convert to list to handle Arabic characters
        training_sentence_tags.append(list(tags))
    except SyntaxError:
        print(f"SyntaxError -Training- in line: {tagged_sentence.strip()}")
        

# Extract sentences and tags from tagged sentences -> Testing Sentences/Tags
sentences_testing, testing_sentence_tags = [], []
for tagged_sentence in testing_sentences:
    try:
        sentence, tags = zip(*ast.literal_eval(tagged_sentence.strip()))
        sentences_testing.append(list(sentence))  # Convert to list to handle Arabic characters
        testing_sentence_tags.append(list(tags))
    except SyntaxError:
        print(f"SyntaxError -Testing- in line: {tagged_sentence.strip()}")
        
        
train_sentences = sentences_training
train_tags = training_sentence_tags

test_sentences = sentences_testing
test_tags = testing_sentence_tags

# Obtaining Unique Words and Tags from NLP Data

In [33]:
# Get unique words and tags
words = get_words(train_sentences)
tags = get_tags(train_tags)

# Creating Word-to-Index and Tag-to-Index Dictionaries for NLP Data

In [34]:
# Create word-to-index and tag-to-index dictionaries
word2index = {w: i + 2 for i, w in enumerate(words)}
word2index['-PAD-'] = 0
word2index['-OOV-'] = 1

tag2index = {t: i + 1 for i, t in enumerate(tags)}
tag2index['-PAD-'] = 0


# Converting Sentences and Tags to Index Sequences for NLP Model Training

In [35]:
# Convert sentences and tags to index sequences
train_sentences_x = get_train_sentences_x(train_sentences, word2index)
test_sentences_x = get_test_sentences_x(test_sentences, word2index)

train_tags_y = get_train_tags_y(train_tags, tag2index)
test_tags_y = get_test_tags_y(test_tags, tag2index)

# Padding Sequences for Uniform Length in NLP Model Training


In [36]:
# Pad sequences to a common length
MAX_LENGTH = max(len(max(train_sentences_x, key=len)), len(max(test_sentences_x, key=len)))

train_sentences_x = pad_sequences(train_sentences_x, maxlen=MAX_LENGTH, padding='post')
test_sentences_x = pad_sequences(test_sentences_x, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')

# Building and Compiling a Bidirectional LSTM Model for Sequence Labeling

In [37]:
from keras.layers import Bidirectional

model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH,)))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(len(tag2index)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

# Training


In [38]:
# Train the model
history = model.fit(train_sentences_x, to_categorical(train_tags_y, len(tag2index)), batch_size=32, epochs=5, verbose=1)

Epoch 1/5
850/850 [==============================] - 386s 451ms/step - loss: 0.1391 - accuracy: 0.9837
Epoch 2/5
850/850 [==============================] - 393s 463ms/step - loss: 0.0446 - accuracy: 0.9921
Epoch 3/5
850/850 [==============================] - 323s 380ms/step - loss: 0.0286 - accuracy: 0.9944
Epoch 4/5
850/850 [==============================] - 347s 409ms/step - loss: 0.0198 - accuracy: 0.9954
Epoch 5/5
850/850 [==============================] - 337s 396ms/step - loss: 0.0144 - accuracy: 0.9961


# Model Anatomy


In [39]:
# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 289, 128)          7299712   
                                                                 
 bidirectional (Bidirection  (None, 289, 256)          263168    
 al)                                                             
                                                                 
 dense (Dense)               (None, 289, 213)          54741     
                                                                 
 activation (Activation)     (None, 289, 213)          0         
                                                                 
Total params: 7617621 (29.06 MB)
Trainable params: 7617621 (29.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Save Model


In [40]:
model.save("BiLSTM_G.h5")

/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Dump Notebook Environment
Dump current kernel and notebook environment to the file below so you can quickly load it next time you open the notebook.

In [ ]:
import dill
dill.dump_session('notebookWorking_env.db')

In [41]:
from sklearn.metrics import confusion_matrix
import numpy as np

predictions = model.predict(test_sentences_x)

predicted_labels = np.argmax(predictions, axis=2)

true_labels_flat = test_tags_y.flatten()
predicted_labels_flat = predicted_labels.flatten()

non_pad_elements = true_labels_flat != 0

conf_matrix = confusion_matrix(true_labels_flat[non_pad_elements], predicted_labels_flat[non_pad_elements])

print("Confusion Matrix:")
print(conf_matrix)


27/27 [==============================] - 4s 109ms/step
Confusion Matrix:
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 3 0]
 [0 0 0 ... 0 0 0]]


In [42]:
from sklearn.metrics import classification_report
from tabulate import tabulate

non_pad_elements = true_labels_flat != 0

report = classification_report(true_labels_flat[non_pad_elements], 
                               predicted_labels_flat[non_pad_elements], 
                               labels=np.unique(predicted_labels_flat[non_pad_elements]))

print("Classification Report:")
# print(report)


array_output = report.split()

for i in range(4, len(array_output)-20, 5):
    id = array_output[i]
    
    desired_value = int(id)
    desired_key = None

    for key, value in tag2index.items():
        if value == desired_value:
            desired_key = key
            break
    array_output[i] = desired_key
    
# print(array_output)
array_output = array_output[9:]

# print(len(array_output))
for i in range(len(array_output) - 1, -1, -1):
    if array_output[i] == "avg":
        array_output[i-1] += "_avg"
        del array_output[i]
# print(array_output)


table = [array_output[i:i+5] for i in range(0, len(array_output), 5)]

print(tabulate(table[0:], headers=['Precision', 'Recall', 'F1-score', 'Support'], tablefmt='pretty'))



Classification Report:
+---------------------------+-----------+--------+----------+---------+
|                           | Precision | Recall | F1-score | Support |
+---------------------------+-----------+--------+----------+---------+
|         B-Artist          |   0.90    |  0.64  |   0.75   |   14    |
|           I-Car           |   0.00    |  0.00  |   0.00   |    0    |
|         B-Author          |   0.29    |  0.08  |   0.12   |   25    |
|         B-Engine          |   0.67    |  0.50  |   0.57   |    4    |
|          I-Time           |   0.83    |  0.05  |   0.10   |   185   |
|          B-Award          |   0.92    |  0.71  |   0.80   |   17    |
|          I-Date           |   0.76    |  0.80  |   0.78   |   235   |
|        B-Religion         |   0.96    |  0.74  |   0.84   |   31    |
|          B-City           |   0.80    |  0.45  |   0.58   |   109   |
|          I-Film           |   0.33    |  0.50  |   0.40   |    2    |
|        I-Currency         |   0.60    |

/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ziadhamwi/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Testing the Model

This code turns the model into a tag-wielding champion! Equipped with a custom function, it takes any sentence and predicts the appropriate tag for each word, like adding punctuation marks to complete a thought. Watch as it analyzes a sample sentence and unveils its predicted tags, revealing the model's real-world potential to understand and structure language on the fly!

In [43]:
# Function to predict tags for a given sentence
def predict_tags(sentence, model, word2index, index2tag):

    sentence_x = [word2index.get(word, word2index['-OOV-']) for word in sentence]

    sentence_x = pad_sequences([sentence_x], maxlen=MAX_LENGTH, padding='post')

    predictions = model.predict(sentence_x)

    predicted_tags = logits_to_tokens(predictions[0], index2tag)

    return predicted_tags

test_sentence = ['كيرلا', 'هي', 'ولاية', 'هندية', 'تقع', 'على', 'ساحل', 'مليبار', 'الهندي', '،', 'أُنشئت', 'في', '1', 'نوفمبر', '1956', 'بعد', 'إقرار', 'قانون', 'إعادة', 'تنظيم', 'الولايات', '،', 'حيث', 'جمعت', 'بين', 'المناطق', 'الناطقة', 'باللغة', 'المالايالامية', 'من', 'مناطق', 'كوشين', 'ومليبار', 'وجنوب', 'كانارا', 'وترافانكور', '،', 'وتبلغ', 'مساحتها', '38,863', 'كيلومتر', 'مربع', '(15,005', 'ميل2)', 'وهي', 'الولاية', 'الهندية', 'الحادية', 'والعشرون', 'من', 'حيث', 'المساحة', '،', 'تحدها', 'ولاية', 'كارناتاكا', 'من', 'الشمال', 'والشمال', 'الشرقي', '،', 'وتاميل', 'نادو', 'من', 'الشرق', 'والجنوب', '،', 'وبحر', 'لكديف', 'من', 'الغرب', '.']



index2tag = {i: t for t, i in tag2index.items()}

predicted_tags = predict_tags(test_sentence, model, word2index, index2tag)

print("Original Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags)

1/1 [==============================] - 0s 55ms/step
Original Sentence: ['كيرلا', 'هي', 'ولاية', 'هندية', 'تقع', 'على', 'ساحل', 'مليبار', 'الهندي', '،', 'أُنشئت', 'في', '1', 'نوفمبر', '1956', 'بعد', 'إقرار', 'قانون', 'إعادة', 'تنظيم', 'الولايات', '،', 'حيث', 'جمعت', 'بين', 'المناطق', 'الناطقة', 'باللغة', 'المالايالامية', 'من', 'مناطق', 'كوشين', 'ومليبار', 'وجنوب', 'كانارا', 'وترافانكور', '،', 'وتبلغ', 'مساحتها', '38,863', 'كيلومتر', 'مربع', '(15,005', 'ميل2)', 'وهي', 'الولاية', 'الهندية', 'الحادية', 'والعشرون', 'من', 'حيث', 'المساحة', '،', 'تحدها', 'ولاية', 'كارناتاكا', 'من', 'الشمال', 'والشمال', 'الشرقي', '،', 'وتاميل', 'نادو', 'من', 'الشرق', 'والجنوب', '،', 'وبحر', 'لكديف', 'من', 'الغرب', '.']
Predicted Tags: ['OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'B-Nationality', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'B-Nationality', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'B-Date', 'I-Date', 'I-Date', 'OUTSIDE', 'OUTSIDE', 'B-Law', 'I-Law', 'I-Law', 'I-Law', 'I-Law', 'OUTSIDE', 'OUTSIDE', 'OUTSIDE', 'OUTS